# Extended repressilator

This repressilator includes additional "steps" - signalling molecules that act as intermediates between repressors.

In [ ]:
pip install -e .

In [ ]:
import loica as lc
import matplotlib.pyplot as plt
import networkx as nx
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def growth_rate(t):
    return 1 # lc.gompertz_growth_rate(t, 0.01, 1, 1, 1)

def biomass(t):
    return 1 #lc.gompertz(t, 0.01, 1, 1, 1)

In [ ]:
k2 = [1, 10, 100, 1000]
alpha = [10**-8, 10**-7, 10**-6, 10**-5, 10**-4, 10**-3, 10**-2, 0.1]

for a in alpha:
    for k in k2:
        metab = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
        genetic_n = lc.GeneticNetwork()
        strain = lc.Strain('Strain', genetic_n, metab)
        # Transcription factors
        laci = lc.Regulator(name='LacI', degradation_rate=1, init_concentration=10**-7)
        tetr = lc.Regulator(name='TetR', degradation_rate=1)
        ci = lc.Regulator(name='cI', degradation_rate=1)

        # Signalling molecules
        c4 = lc.Regulator(name='C4', degradation_rate=1)
        scb2 = lc.Regulator(name='SCB2', degradation_rate=1)
        c14 = lc.Regulator(name='C14', degradation_rate=1)

        genetic_n.add_regulator([tetr, laci, ci, c4, scb2, c14])

        rhli = lc.Regulator(name='RhlI', degradation_rate=1)
        scbb = lc.Regulator(name='ScbB', degradation_rate=1)
        cini = lc.Regulator(name='CinI', degradation_rate=1)
        genetic_n.add_regulator([rhli, scbb, cini])

        mcherry = lc.Reporter(name='mCherry', degradation_rate=1, color='red') 
        cyan = lc.Reporter(name='CFP', degradation_rate=1, color='blue') 
        mven = lc.Reporter(name='mVenus', degradation_rate=1, color='yellow') 

        genetic_n.add_reporter([mcherry, cyan, mven])

        #First extended node
        c4_laci = lc.Hill1(name='Prhl', input=c4, output=[laci, cyan], alpha=[0, a], K=10**-5, n=1.255)
        laci_not_scbb = lc.Hill1(name='Plac', input=laci, output=scbb,  alpha=[10**-4, 0], K=10**-8, n=2)
        scbb_scb2 = lc.Producer(name='ScbB', enzyme=scbb, product=scb2, k2=k)

        # Second
        scb2_tetr = lc.Hill1(name='Pscb', input=scb2, output=[tetr, mcherry], alpha=[0, a], K=10**-5, n=1.255)
        tetr_not_cini = lc.Hill1(name='Ptet', input=tetr, output=cini,  alpha=[10**-4, 0], K=10**-8, n=2)
        cini_c14 = lc.Producer(name='CinI', enzyme=cini, product=c14, k2=k)

        # Third
        c14_ci = lc.Hill1(name='Pcin', input=c14, output=[ci, mven], alpha=[0, a], K=10**-5, n=1.255)
        ci_not_rhli = lc.Hill1(name='PcI', input=ci, output=rhli,  alpha=[10**-4, 0], K=10**-8, n=2)
        rhli_c4 = lc.Producer(name='RhlI', enzyme=rhli, product=c4, k2=k)

        genetic_n.add_operator([c4_laci, laci_not_scbb, scb2_tetr, tetr_not_cini, c14_ci, ci_not_rhli]) 
        genetic_n.add_impactor([scbb_scb2, cini_c14, rhli_c4])

        sample = lc.Sample(strain=strain)
        assay = lc.Assay([sample], 
                        n_measurements=100, 
                        interval=0.24,
                        name='Extended repressilator',
                        description='Simulated expression of extended repressilator strain'
                        )
        assay.run(substeps=10, mode="track_all")

        m = assay.measurements
        fig, ax = plt.subplots()
        fig1, ax1 = plt.subplots()
        fig2, ax2 = plt.subplots()
        fig3, ax3 = plt.subplots()
        m[m.Signal=='mCherry in Strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
        m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax)
        m[m.Signal=='mVenus in Strain'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax)
        m[m.Signal=='LacI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
        m[m.Signal=='TetR in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
        m[m.Signal=='cI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
        m[m.Signal=='C4 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
        m[m.Signal=='SCB2 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
        m[m.Signal=='C14 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
        m[m.Signal=='RhlI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
        m[m.Signal=='ScbB in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
        m[m.Signal=='CinI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
        ax.legend(['mCherry', 'CFP', 'mVenus'])
        ax.set_xlabel('Time (h)')
        ax.set_ylabel('Concentration (M)')
        ax1.legend(['LacI', 'TetR', 'cI'])
        ax1.set_xlabel('Time (h)')
        ax1.set_ylabel('Concentration (M)')
        ax2.legend(['C4', 'SCB2', 'C14'])
        ax2.set_xlabel('Time (h)')
        ax2.set_ylabel('Concentration (M)')
        ax3.legend(['RhlI', 'ScbB', 'CinI'])
        ax3.set_xlabel('Time (h)')
        ax3.set_ylabel('Concentration (M)')
